<a href="https://colab.research.google.com/github/mariusblarsen/ae-attack-pipeline/blob/SAA-attack/SAA_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3. Generate adversarial example

## Sparse-Adversarial-Attack

In [1]:
!git clone https://github.com/mariusblarsen/ae-attack-pipeline

Cloning into 'ae-attack-pipeline'...
remote: Enumerating objects: 1492, done.
remote: Counting objects: 100% (1492/1492), done.
remote: Compressing objects: 100% (950/950), done.
remote: Total 1492 (delta 525), reused 1488 (delta 521), pack-reused 0
Receiving objects: 100% (1492/1492), 15.04 MiB | 21.66 MiB/s, done.
Resolving deltas: 100% (525/525), done.


In [2]:
%cd ae-attack-pipeline

/content/ae-attack-pipeline


In [3]:
!git pull origin SAA-attack

From https://github.com/mariusblarsen/ae-attack-pipeline
 * branch            SAA-attack -> FETCH_HEAD
Updating ec0e49c..b38b6e1
Fast-forward
 .gitignore                                         |    5 +
 .ipynb_checkpoints/SAA-demo-checkpoint.ipynb       |  700 ++++++
 SAA-demo.ipynb                                     |  726 ++++++
 Tianchi04/README.md                                |   19 +
 Tianchi04/assets/1.jpg                             |  Bin 0 -> 247602 bytes
 Tianchi04/attack_1.py                              |  270 ++
 Tianchi04/attack_2.py                              |  289 +++
 Tianchi04/attack_3.py                              |  318 +++
 Tianchi04/eval.py                                  |  174 ++
 Tianchi04/improve_1.py                             |  300 +++
 Tianchi04/improve_2.py                             |  295 +++
 Tianchi04/improve_3.py                             |  297 +++
 Tianchi04/infer.py                                 |   62 +
 .../mmdetection/.dev_scrip

In [4]:
!git checkout -b SAA-attack

Switched to a new branch 'SAA-attack'


In [7]:
# Download already instal
from google.colab import drive
drive.mount('/content/gdrive')
!pip install --upgrade --force-reinstall `cat /content/gdrive/My\ Drive/colab_installed.txt`

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
ERROR: Invalid requirement: '@'


In [9]:
!pip install -r Tianchi04/requirements.txt

from google.colab import drive
drive.mount('/content/gdrive')
!pip freeze --local > /content/gdrive/My\ Drive/colab_installed.txt

     |████████████████████████████████| 324 kB 5.2 MB/s 
     |████████████████████████████████| 1.2 MB 37.2 MB/s 
     |████████████████████████████████| 185 kB 46.0 MB/s 
  Created wheel for mmcv: filename=mmcv-1.3.14-py2.py3-none-any.whl size=471262 sha256=941b9be0f524f83ded57c50a583c2637c78f3b3c4fe5cf4c42ff917744efd602
  Stored in directory: /root/.cache/pip/wheels/ed/31/c0/8a9c014d146ea76332e5463eba0af8df69d18d49f22878c1ae
  Created wheel for terminaltables: filename=terminaltables-3.1.0-py3-none-any.whl size=15354 sha256=9c114886e1614dbd96f8bcfdd295171177e3ba923d8674d4e6a4d26cd585416b
  Stored in directory: /root/.cache/pip/wheels/ba/ad/c8/2d98360791161cd3db6daf6b5e730f34021fc9367d5879f497
Successfully built mmcv terminaltables
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [33]:
%cd Tianchi04
!ls

/content/ae-attack-pipeline/Tianchi04
assets	     eval.py	   infer.py	pixel.json	  select1000_new_p
attack_1.py  improve_1.py  mmdetection	README.md	  tool
attack_2.py  improve_2.py  models.py	requirements.txt  utils
attack_3.py  improve_3.py  output_data	select1000_new


In [35]:
%cd gdrive/MyDrive/Studie/Prosjekttoppgave/

/content
/content/gdrive/MyDrive/Studie/Prosjekttoppgave


In [36]:
%cp -av Colab\ assets checkpoints

'Colab assets' -> 'checkpoints'
'Colab assets/yolov4.cfg' -> 'checkpoints/yolov4.cfg'
'Colab assets/yolov4.weights' -> 'checkpoints/yolov4.weights'
'Colab assets/yolov4.pth' -> 'checkpoints/yolov4.pth'
'Colab assets/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth' -> 'checkpoints/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth'


In [37]:
%mv checkpoints ../../../../ae-attack-pipeline/Tianchi04/

In [38]:
%cd ../../../../ae-attack-pipeline/Tianchi04/
!ls checkpoints

/content/ae-attack-pipeline/Tianchi04


In [ ]:
# Upload ./checkpoints files
%cd checkpoints
from google.colab import files
uploaded = files.upload()
%cd ..

/content/ae-attack-pipeline/Tianchi04/checkpoints


Saving faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth to faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth
Saving yolov4.cfg to yolov4.cfg
Saving yolov4.pth to yolov4.pth
Saving yolov4.weights to yolov4.weights
/content/ae-attack-pipeline/Tianchi04


In [ ]:
!rm -rf ./select1000_new_p/.ipynb_checkpoints/

In [45]:
!pip install mmcv-full

     |████████████████████████████████| 324 kB 5.4 MB/s 
  Created wheel for mmcv-full: filename=mmcv_full-1.3.14-cp37-cp37m-linux_x86_64.whl size=31464497 sha256=38d0d367b392ca3ca73837e222fbf9ff9750dbcb478ba5a83ec9d39191f16efa
  Stored in directory: /root/.cache/pip/wheels/5e/54/62/69c99dc3c9937bca64126f81cbe315ae6c8e6e98c43fa7392d
Successfully built mmcv-full


In [49]:
!python eval.py

100% 1/1 [00:02<00:00,  2.34s/it]
convolution havn't activate linear
convolution havn't activate linear
convolution havn't activate linear
  0% 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
100% 1/1 [00:00<00:00,  1.71it/s]
Overall score:  0.0
Saving into whitebox_yolo_overall_score.json...
/usr/local/lib/python3.7/dist-packages/mmdet/core/anchor/builder.py:17: UserWarning: ``build_anchor_generator`` would be deprecated soon, please use ``build_prior_generator`` 
  '``build_anchor_generator`` would be deprecated soon, please use '
Use load_from_local loader
  0% 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-pac

In [ ]:
!rm -rf ../images/.ipynb_checkpoints/

In [ ]:
!python attack_3.py

/usr/local/lib/python3.7/dist-packages/mmdet/core/anchor/builder.py:17: UserWarning: ``build_anchor_generator`` would be deprecated soon, please use ``build_prior_generator`` 
  '``build_anchor_generator`` would be deprecated soon, please use '
Use load_from_local loader
device
cuda:0
6.jpg 2483
/usr/local/lib/python3.7/dist-packages/mmdet/apis/inference.py:73: UserWarning: `LoadImage` is deprecated and will be removed in future releases. You may use `LoadImageFromWebcam` from `mmdet.datasets.pipelines.` instead.
  warnings.warn('`LoadImage` is deprecated and will be removed in '
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
/usr/loca

# 5. Attack

In [ ]:
# Image to predict and display
ATTACK_EPS = 0.15  # Used to determine image in attack
ADV_NAME = IMAGE_NAME.split(".")[0] + "-" + str(ATTACK_EPS) + ".png"
ADV_PATH = os.path.join(paths['ADV_PATH'], ADV_NAME)

In [ ]:
from utils.misc import load_image_into_numpy_array

adv_np = load_image_into_numpy_array(ADV_PATH)

# Visualization params
MAX_BBOXES = 200
THRESHOLD = 0.3

input_tensor = tf.convert_to_tensor(np.expand_dims(adv_np, 0), dtype=tf.float32)
detections, predictions_dict, shapes = detect_fn(input_tensor)

num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy()
              for key, value in detections.items()}
detections['num_detections'] = num_detections

# detection_classes should be ints.
detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

label_id_offset = 1
adv_np_with_detections = adv_np.copy()
viz_utils.visualize_boxes_and_labels_on_image_array(
            adv_np_with_detections,
            detections['detection_boxes'],
            detections['detection_classes']+label_id_offset,
            detections['detection_scores'],
            category_index,
            use_normalized_coordinates=True,
            max_boxes_to_draw=MAX_BBOXES,
            min_score_thresh=THRESHOLD,
            agnostic_mode=False)
plt.figure()
plt.imshow(adv_np_with_detections)
plt.title("Adversarial example with epsilon = {}".format(ATTACK_EPS))
plt.show()
plt.imshow(image_np_with_detections)
plt.title("Original image")
plt.show()